Example 1: Matching up logfiles with recipe lists
=========
Extracting a time-ordered list of processes that occurred during deposition from an AJA logfile. In order to run this, you'll have to have the required directories of AJA logfiles, recipe files, and job files.

In [3]:
#Add the AJA repository parent directory to $PATH
import sys
sys.path.append('/Users/fcarter/Documents/Code/Python/')

import glob
import pprint

from AJA_tools import recipe_parser as rp

Specify file locations
---------------------
The easiest thing is to put all the recipe files (`.rcp`) in one folder and all the job files (`.ajp`) in another. I did this by opening a terminal and typing::

    mkdir recipes
    mv *.rcp recipes
    mkdir jobs
    mv *.ajp jobs

In [6]:
recipe_files_dir = './AJA_Metals/recipes/'
job_files_dir = './AJA_Metals/jobs/'

Parse files and generate database of jobs
---------------------------------------
This function takes in a folder of `.ajp` files and builds a dictionary of jobs that matches job names to job file locations.

In [7]:
#Build up the dictionary of jobs
jobs_dict = rp.build_jobs_dict(job_files_dir)

#Print the first couple of job names
print("The first five jobs are:")
pprint.pprint(list(jobs_dict.keys())[0:5])

The first five jobs are:
['1 9',
 'AlMn950_100nm_Ti10nm_Au20nm_test ',
 'bias test w4',
 'CP 300nm Nb_250W_30W_RF_8mtorr',
 'CP 4X 30 min Nb deposition to improve vacuum_wait_2_4&_6hrs']


Get the path to the right job file
--------------------------------
`get_job` takes in a logfile path and a dictionary of jobs (or path to a jobs folder) and returns both the job name and the job file.

In [8]:
#Specify the logfile of interest
log_file_path = './AJA_Metals/datalog/CP LU1_3_4_5 TES Ti+Au_200nm+20nm 5minRFclean_Wait5min_1252016_ 102948 AM.dlg'

job_name, job_file_path = rp.get_job(log_file_path, jobs_dict)
print("Job name is: "+job_name)
print("Job file is: "+job_file_path)
with open(job_file_path, 'r') as f:
    print(repr(f.read()))

Job name is: CP LU1_3_4_5 TES Ti+Au_200nm+20nm 5minRFclean_Wait5min
Job file is: ./AJA_Metals/jobs/CP LU1_3_4_5 TES Ti+Au_200nm+20nm 5minRFclean_Wait5min.ajp
'\x00\x00\x00<\x00\x00\x00\x06Load 1\x00\x00\x00\twait 5min\x00\x00\x00!CP Rot Gas ON  30W Bias Plasma On\x00\x00\x00\x10CP RF Clean 5min\x00\x00\x00\x11CP Gases OFF 1min\x00\x00\x00\twait 5min\x00\x00\x00 SL Rot Gas ON & Strike Ti Plasma\x00\x00\x00\x14CP Ti Presputt 3 min\x00\x00\x00-CP Ti sputter 200nm + Au presputter last 1min\x00\x00\x00\x15SL Ti OFF + Au strike\x00\x00\x00\x1bSL Au sputter 20nm 3_2mTorr\x00\x00\x00\tSL Au OFF\x00\x00\x00\x10SL Rot Gases OFF\x00\x00\x00\twait 5min\x00\x00\x00\x08Unload 1\x00\x00\x00\x05load3\x00\x00\x00\twait 5min\x00\x00\x00!CP Rot Gas ON  30W Bias Plasma On\x00\x00\x00\x10CP RF Clean 5min\x00\x00\x00\x11CP Gases OFF 1min\x00\x00\x00\twait 5min\x00\x00\x00 SL Rot Gas ON & Strike Ti Plasma\x00\x00\x00\x14CP Ti Presputt 3 min\x00\x00\x00-CP Ti sputter 200nm + Au presputter last 1min\x00\x00\x0

Extract the recipe from the job file
----------------------------------
Since the job files all have a specific format, passing in the recipe files directory path is optional. However, if that path is passed in, the function will check if each recipe step exists as a recipe file in the directory.

In [9]:
recipe_dict = rp.get_recipe(job_file_path, recipe_folder_path=recipe_files_dir)
pprint.pprint(recipe_dict)

{'info': 'raw_recipe format is: (string index, string, recipe exists?)',
 'job_name': 'CP LU1_3_4_5 TES Ti+Au_200nm+20nm 5minRFclean_Wait5min',
 'raw_job': '\x00\x00\x00<\x00\x00\x00\x06Load 1\x00\x00\x00\twait '
            '5min\x00\x00\x00!CP Rot Gas ON  30W Bias Plasma '
            'On\x00\x00\x00\x10CP RF Clean 5min\x00\x00\x00\x11CP Gases OFF '
            '1min\x00\x00\x00\twait 5min\x00\x00\x00 SL Rot Gas ON & Strike Ti '
            'Plasma\x00\x00\x00\x14CP Ti Presputt 3 min\x00\x00\x00-CP Ti '
            'sputter 200nm + Au presputter last 1min\x00\x00\x00\x15SL Ti OFF '
            '+ Au strike\x00\x00\x00\x1bSL Au sputter 20nm '
            '3_2mTorr\x00\x00\x00\tSL Au OFF\x00\x00\x00\x10SL Rot Gases '
            'OFF\x00\x00\x00\twait 5min\x00\x00\x00\x08Unload '
            '1\x00\x00\x00\x05load3\x00\x00\x00\twait 5min\x00\x00\x00!CP Rot '
            'Gas ON  30W Bias Plasma On\x00\x00\x00\x10CP RF Clean '
            '5min\x00\x00\x00\x11CP Gases OFF 1min\x00\x00\x

Check for missing job files
-------------------------
Now that we've gone through all this, we may as well check to see if any of the logfiles don't have jobs associated with them.

In [10]:
logs_dict = {}

logfiles = glob.glob('./AJA_Metals/datalog/*')
for log in logfiles:
    job_name, job_file = rp.get_job(log, jobs_dict)
    if job_file is None:
        print("Bad job: "+job_name)
    else:
        logs_dict[log] = rp.get_recipe(job_file, recipe_files_dir)
        

Bad job: CP LU1_3_4_5_6 TES 1&3&4 Ti4+Au_200+30nm 5&6 Ti4+Au+Ti4+Au_5+5+150+20nm 5minRFclean_Wait10min_temp
Bad job: load-unload1-9
Bad job: load-unload1-9
Bad job: load-unload1-9
Bad job: load-unload1-9
Bad job: load-unload1-9
Bad job: SP LU1_3_4_5_6 Resistor Ti+Au_40+7nm_temp
Bad job: SP LU1_3_4_5_6 Resistor Ti+Au_40+7nm_temp
Bad job: SP LU1_3_4_5_6 Resistor Ti+Au_50+7nm_temp
Bad job: SP LU1_3_4_5_6 Resistor Ti+Au_50+7nm_temp
Bad job: SP LU1_3_4_5_6 Resistor Ti+Au_50+7nm_temp
Bad job: SP LU1_3_4_5_6 Resistor Ti+Au_50+7nm_temp
Bad job: SP LU1_3_4_5_6 TES Ti4+Au+Ti4+Au_5+5+160+20nm 5minRFclean_Wait10min_Temp
Bad job: SP LU1_3_4_5_6 TES Ti4+Au+Ti4+Au_5+5+175+20nm 5minRFclean_Wait10min_temp
Bad job: SP LU1_3_4_5_6 TES Ti4+Au+Ti4+Au_5+5+175+20nm 5minRFclean_Wait10min_temp
Bad job: 
Bad job: test
Bad job: TiN 500A 400C N2 Sweep
Bad job: TiN 500A 400C N2 Sweep
Bad job: TiN 500A 400C N2 Sweep


/Users/fcarter/Documents/Code/Python/AJA_tools/recipe_parser.py:103: RecipeWarning: No matching job found!
  warnings.warn("No matching job found!", RecipeWarning)
